In [1]:
import numpy as np
import pandas as pd
import dtale
import pandas_profiling as pp
from IPython.display import Javascript
from plotly.offline import iplot, init_notebook_mode

In [2]:
# load the dataset
accident_victims_2019_df = pd.read_parquet("../../data/silver/accident_victims_2019_df.parquet")

In [3]:
accident_victims_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85607 entries, 0 to 85606
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DT_DAY                     85607 non-null  datetime64[ns]
 1   DT_HOUR                    85607 non-null  Int64         
 2   CD_DAY_OF_WEEK             85607 non-null  Int64         
 3   TX_DAY_OF_WEEK_DESCR_FR    85607 non-null  string        
 4   TX_DAY_OF_WEEK_DESCR_NL    85607 non-null  string        
 5   MS_VICT                    85607 non-null  Int64         
 6   MS_VIC_OK                  85607 non-null  Int64         
 7   MS_SLY_INJ                 85607 non-null  Int64         
 8   MS_SERLY_INJ               85607 non-null  Int64         
 9   MS_DEAD_30_DAYS            85607 non-null  Int64         
 10  CD_BUILD_UP_AREA           85607 non-null  Int64         
 11  TX_BUILD_UP_AREA_DESCR_NL  85607 non-null  string        
 12  TX_B

In [4]:
# remove any duplicate values
accident_victims_2019_df = accident_victims_2019_df.drop_duplicates().reset_index(drop=True)

# keep only the victim type 'bicycle' users

In [5]:
accident_victims_2019_df['TX_VICT_TYPE_DESCR_NL'].unique()

<StringArray>
[    'Bromfietser',      'Bestuurder',      'Voetganger',       'Passagier',
         'Fietser',    'Motorfietser', 'Autres victimes',        'Onbekend']
Length: 8, dtype: string

In [6]:
# remove all other records, keep only the victim type 'bicycle' type 
accident_victims_2019_df = accident_victims_2019_df[(accident_victims_2019_df['TX_VICT_TYPE_DESCR_NL'] == 'Motorfietser')]

In [7]:
# select only a few important columns (date, vilage, ...)
accident_per_day_2019_df = accident_victims_2019_df[["DT_DAY", "TX_BUILD_UP_AREA_DESCR_NL", "CD_ROAD_USER_TYPE", "TX_ROAD_TYPE_DESCR_NL", "TX_LIGHT_COND_DESCR_NL", "TX_AGE_CLS_DESCR_NL", "TX_MUNTY_DESCR_NL"]]

In [8]:
accident_per_day_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2828 entries, 8 to 85579
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DT_DAY                     2828 non-null   datetime64[ns]
 1   TX_BUILD_UP_AREA_DESCR_NL  2828 non-null   string        
 2   CD_ROAD_USER_TYPE          2828 non-null   Int64         
 3   TX_ROAD_TYPE_DESCR_NL      2828 non-null   string        
 4   TX_LIGHT_COND_DESCR_NL     2828 non-null   string        
 5   TX_AGE_CLS_DESCR_NL        2828 non-null   string        
 6   TX_MUNTY_DESCR_NL          2828 non-null   string        
dtypes: Int64(1), datetime64[ns](1), string(5)
memory usage: 179.5 KB


In [9]:
accident_per_day_2019_df

,DT_DAY,TX_BUILD_UP_AREA_DESCR_NL,CD_ROAD_USER_TYPE,TX_ROAD_TYPE_DESCR_NL,TX_LIGHT_COND_DESCR_NL,TX_AGE_CLS_DESCR_NL,TX_MUNTY_DESCR_NL
8,2019-11-13,Buiten bebouwde kom,14,Autosnelweg,Bij klaarlichte dag,55 tot 59 jaar,Aartselaar
18,2019-10-07,Buiten bebouwde kom,13,Autosnelweg,Bij klaarlichte dag,20 tot 24 jaar,Aartselaar
90,2019-09-26,Buiten bebouwde kom,13,Gewestweg,"Nacht, ontstoken openbare verlichting",30 tot 34 jaar,Aartselaar
95,2019-08-31,Buiten bebouwde kom,14,Gemeenteweg,Bij klaarlichte dag,45 tot 49 jaar,Aartselaar
106,2019-06-13,Binnen bebouwde kom,14,Gemeenteweg,Bij klaarlichte dag,45 tot 49 jaar,Aartselaar
...,...,...,...,...,...,...,...
85529,2019-02-17,Buiten bebouwde kom,14,Gewestweg,Bij klaarlichte dag,60 tot 64 jaar,Walcourt
85542,2019-10-29,Buiten bebouwde kom,14,Gewestweg,"Nacht, geen openbare verlichting",40 tot 44 jaar,Walcourt
85562,2019-03-30,Binnen bebouwde kom,13,Gewestweg,Bij klaarlichte dag,30 tot 34 jaar,Viroinval
85575,2019-06-15,Buiten bebouwde kom,14,Gewestweg,Bij klaarlichte dag,50 tot 54 jaar,Viroinval


## count is the right function to use
but, we need the number of incidents that day

In [10]:
# sort date first, then sort on vilagename, then group these two and count() the number of incidents that occured
# then create a new dataframe and sort the results with the most accidents on a given day first 
groupby_accident_per_day_2019_df = accident_per_day_2019_df.groupby(['DT_DAY', 'TX_MUNTY_DESCR_NL'])['TX_ROAD_TYPE_DESCR_NL'].count().to_frame(name='nr_of_accidents').reset_index()
groupby_accident_per_day_2019_df = groupby_accident_per_day_2019_df.sort_values(by=['nr_of_accidents', 'DT_DAY', 'TX_MUNTY_DESCR_NL'], ascending=[False, True, False])
groupby_accident_per_day_2019_df

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents
2421,2019-11-13,Brussel,5
780,2019-05-14,Antwerpen,4
915,2019-05-27,Antwerpen,4
1737,2019-08-23,Beveren (Sint-Niklaas),4
1930,2019-09-11,Brussel,4
...,...,...,...
2635,2019-12-30,Bergen,1
2634,2019-12-30,Awans,1
2640,2019-12-31,Eigenbrakel,1
2639,2019-12-31,Awans,1


In [11]:
# this was to check and verify the correctness of the results - ALL GOOD
groupby_accident_per_day_2019_df[groupby_accident_per_day_2019_df["TX_MUNTY_DESCR_NL"] == 'Antwerpen'].sort_values(by=['DT_DAY'], ascending=[True])

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents
5,2019-01-03,Antwerpen,1
8,2019-01-04,Antwerpen,1
22,2019-01-08,Antwerpen,1
26,2019-01-09,Antwerpen,1
48,2019-01-16,Antwerpen,1
...,...,...,...
2593,2019-12-18,Antwerpen,1
2600,2019-12-19,Antwerpen,2
2603,2019-12-20,Antwerpen,1
2610,2019-12-22,Antwerpen,1


# so now, we calculate the accidents/cyclists RATIO

In [12]:
# load population values
village_populations_wiki_df = pd.read_parquet("../../data/silver/village_populations_wiki_df.parquet")

In [13]:
# village_populations_wiki_df.columns = ["Village", "Population", "km2_size"]

In [14]:
village_populations_wiki_df

,Village,Population,km2_size
0,Antwerpen,446525,"204,51"
1,Gent,224180,"156,18"
2,Charleroi,200827,"102,08"
3,Luik,185639,"69,39"
4,Brussel,133859,"32,61"
...,...,...,...
576,Martelange,1428,"29,67"
577,Herbeumont,1434,"58,81"
578,Daverdisse,1360,"56,40"
579,Mesen,964,"3,58"


In [15]:
village_populations_wiki_df['km2_size'] = village_populations_wiki_df['km2_size'].str[:-3:].astype(int)

In [16]:
village_populations_wiki_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Village     581 non-null    string
 1   Population  581 non-null    Int64 
 2   km2_size    581 non-null    int64 
dtypes: Int64(1), int64(1), string(1)
memory usage: 14.3 KB


# now we calculate the ratio

In [17]:
groupby_accident_per_day_2019_df = groupby_accident_per_day_2019_df.replace({'TX_MUNTY_DESCR_NL':{
    'Sint-Niklaas (Sint-Niklaas)': 'Sint-Niklaas',
    'Halle (Halle-Vilvoorde)': 'Halle',
    'Beveren (Sint-Niklaas)': 'Beveren',
    'Tielt (Tielt)': 'Tielt',
    'Vorst (Brussel-Hoofdstad)': 'Vorst',
    'Kapellen (Antwerpen)': 'Kapellen',
    'Hamme (Dendermonde)': 'Hamme',
    'Nieuwerkerken (Hasselt)': 'Nieuwerkerken',
    'Machelen (Halle-Vilvoorde)': 'Machelen',
    'Hove (Antwerpen)': 'Hove',
    'Celles (Doornik)': 'Celles',
    'Perwijs (Nijvel)': 'Perwijs',
    'Saint-Nicolas (Luik)': 'Saint-Nicolas',
    'Aalst (Aalst)': 'Aalst',
    'Moerbeke (Gent)': 'Moerbeke'}})

In [18]:
village_populations_wiki_df = village_populations_wiki_df.replace({'Village':{
    'Pelt*': 'Pelt',
    'Deinze*': 'Deinze',
    'Kruisem*': 'Kruisem',
    'Aalter*': 'Aalter',
    'Belœil': 'Beloeil',
    'Lievegem*': 'Lievegem',
    "Mont-de-l'Enclus": 'Mont-de-l’Enclus',
    'Puurs-Sint-Amands*': 'Puurs-Sint-Amands',
    'Court-Saint-Étienne': 'Court-Saint-Etienne',
    'Éghezée': 'Eghezée',
    'Oudsbergen*': 'Oudsbergen',
    'Estaimpuis*': 'Estaimpuis',
    "Fontaine-l'Evêque": 'Fontaine-l’Evêque',
    'Gembloers': 'Gembloux',
    '’s Gravenbrakel*': '’s Gravenbrakel',
    'Erezée*': 'Erezée',
    'Ecaussines': 'Ecaussinnes',
    'Blegny*': 'Blegny',
    'Plombières*': 'Plombières'}})

In [19]:
# merge join on village
merged_accidents_df = groupby_accident_per_day_2019_df.merge(village_populations_wiki_df, left_on='TX_MUNTY_DESCR_NL', right_on='Village', how='left')

In [20]:
# remove Nan records
merged_accidents_df = merged_accidents_df[~(merged_accidents_df['Population'].isna())]

# calculate the ratio itself

### (nr of incidents / population ) * city size in km

In [21]:
merged_accidents_df.loc[:, 'accident_village_ratio'] = merged_accidents_df['nr_of_accidents'] / merged_accidents_df['Population']

In [22]:
merged_accidents_df.loc[:, 'accident_village_size_ratio'] = merged_accidents_df['accident_village_ratio'] * merged_accidents_df['km2_size']

In [23]:
merged_accidents_df

,DT_DAY,TX_MUNTY_DESCR_NL,nr_of_accidents,Village,Population,km2_size,accident_village_ratio,accident_village_size_ratio
0,2019-11-13,Brussel,5,Brussel,133859,32.0,0.000037,0.001195
1,2019-05-14,Antwerpen,4,Antwerpen,446525,204.0,0.000009,0.001827
2,2019-05-27,Antwerpen,4,Antwerpen,446525,204.0,0.000009,0.001827
3,2019-08-23,Beveren,4,Beveren,44977,150.0,0.000089,0.01334
4,2019-09-11,Brussel,4,Brussel,133859,32.0,0.00003,0.000956
...,...,...,...,...,...,...,...,...
2636,2019-12-30,Bergen,1,Bergen,90935,146.0,0.000011,0.001606
2637,2019-12-30,Awans,1,Awans,8270,27.0,0.000121,0.003265
2638,2019-12-31,Eigenbrakel,1,Eigenbrakel,35259,52.0,0.000028,0.001475
2639,2019-12-31,Awans,1,Awans,8270,27.0,0.000121,0.003265


In [24]:
accident_village_ratio_median_cutoff = merged_accidents_df['accident_village_ratio'].median()
accident_village_ratio_median_cutoff

3.8954462233648865e-05

In [25]:
accident_village_size_ratio_median_cutoff = merged_accidents_df['accident_village_size_ratio'].median()
accident_village_size_ratio_median_cutoff

0.0014987216785682799

In [27]:
result = merged_accidents_df.groupby(['Village'])['accident_village_size_ratio'].sum().to_frame(name='ratio').reset_index()
result = result.sort_values(by=['ratio', 'Village'], ascending=[False, True])
result.head(10)

,Village,ratio
416,Stoumont,0.26314
148,Froidchapelle,0.259427
448,Vresse-sur-Semois,0.251601
88,Chimay,0.222051
65,Bouillon,0.219239
251,La Roche-en-Ardenne,0.214755
322,Nassogne,0.163235
118,Durbuy,0.157322
82,Cerfontaine,0.134056
211,Houyet,0.110083


In [28]:
s = result['ratio']
q1 = s.quantile(0.25)
q3 = s.quantile(0.75)
iqr = q3 - q1
iqr_lower = q1 - 1.5 * iqr
iqr_upper = q3 + 1.5 * iqr
outliers = dict(s[(s < iqr_lower) | (s > iqr_upper)])

In [29]:
result.iloc[list(outliers.keys())]

,Village,ratio
119,Ecaussinnes,0.003534
485,Zuienkerke,0.017204
63,Bornem,0.002257
61,Borgloon,0.025208
313,Momignies,0.033151
470,Wommelgem,0.009947
261,Lendelede,0.007088
171,Halle,0.020918
469,Wingene,0.027163
235,Knokke-Heist,0.011826


In [30]:
chart_data = pd.concat([
    result.iloc[list(outliers.keys())]['Village'],
    result.iloc[list(outliers.keys())]['ratio'],
], axis=1)
chart_data = chart_data.sort_values(['Village'])
chart_data = chart_data.rename(columns={'Village': 'x'})
chart_data = chart_data.dropna()

import plotly.graph_objs as go

charts = []
charts.append(go.Bar(
    x=chart_data['x'],
    y=chart_data['ratio']
))
figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'stack',
    'legend': {'orientation': 'h'},
    'title': {'text': 'Outliers (highest & lowest)'},
    'xaxis': {'title': {'text': 'Cities'}},
    'yaxis': {'title': {'text': 'ratio of motorcyclists'}, 'type': 'linear'}
}))

In [33]:
init_notebook_mode(connected=True)
for chart in charts:
    chart.pop('id', None) # for some reason iplot does not like 'id'
iplot(figure)

In [32]:
figure.write_image("../../data/plot/accident_ratio_motorcyclists.png")